In [1]:
import numpy as np
import pandas as pd

shops = pd.read_csv('input/shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [2]:
categs = pd.read_csv('input/item_categories.csv')
categs.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [3]:
items = pd.read_csv('input/items.csv', index_col=1)
#items.set_index('item_id')
items.head()

,item_name,item_category_id
item_id,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40
1,!ABBYY FineReader 12 Professional Edition Full...,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40
4,***КОРОБКА (СТЕКЛО) D,40


In [4]:
def get_categ_id_for_item(item_id):
    return items.loc[item_id]['item_category_id']

get_categ_id_for_item(33)

37

In [5]:
transactions = pd.read_csv('input/sales_train_v2.csv')
print(transactions.shape)
transactions.head()
#transactions['date_block_num'].max()

(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [8]:
test_data = pd.read_csv('input/test.csv', index_col=0)
print(test_data.shape)
test_data.head()

(214200, 2)


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [9]:
monthly_totals = transactions.groupby(['date_block_num', 'shop_id', 'item_id'],as_index=False)[['item_cnt_day']].sum()
print(monthly_totals.shape)
monthly_totals.head()

(1609124, 4)


,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [ ]:
indexed_monthly_totals = transactions.groupby(['date_block_num', 'shop_id', 'item_id'])[['item_cnt_day']].sum()
indexed_monthly_totals.head()

In [20]:
#divide into smaller files
monthly_totals_0 = monthly_totals.loc[(monthly_totals['shop_id'] >= 0) & (monthly_totals['shop_id'] < 10)]
print(monthly_totals_0.shape)
monthly_totals_0.head()

(172338, 4)


,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [21]:
#save file
#monthly_totals.to_csv('input/monthly_totals.csv', index=False, columns=['date_block_num','shop_id','item_id','item_cnt_day'])
monthly_totals_0.to_csv('input/monthly_totals_0.csv', index=False)

In [38]:
training_data = pd.read_csv('input/monthly_totals_0.csv')
training_data.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [44]:
def get_total_for_prev_month(date_block_num, shop_id, item_id, prev_month_index):
    target_date_block_num = date_block_num + prev_month_index
    if target_date_block_num < 0:
        return np.nan
    elif target_date_block_num > 33:
        return np.nan
    
    df = monthly_totals_0
    found = df.loc[(df['date_block_num'] == target_date_block_num) & (df['shop_id'] == shop_id) & (df['item_id'] == item_id)]
    if found.empty:
        return 0
    else:
        return found['item_cnt_day'].iloc[0]
    
total = get_total_for_prev_month(0,0,43,0)
print(total)

1.0


In [45]:
def get_prev_month_total_for_row(row, prev_month_index):
    return get_total_for_prev_month(row['date_block_num'],row['shop_id'],row['item_id'],prev_month_index)

training_data['item_cnt_tm1'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, -1), axis=1)
training_data['item_cnt_tm2'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, -2), axis=1)
training_data['item_cnt_tm12'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, -12), axis=1)
training_data['item_cnt_tp1'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, 1), axis=1)
training_data.loc[training_data['item_cnt_tp1'] > 0.0]

,date_block_num,shop_id,item_id,item_cnt_day,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12,item_cnt_tp1,categ_id
0,0,0,32,6.0,NaN,NaN,NaN,10.0,40
1,0,0,33,3.0,NaN,NaN,NaN,3.0,37
2,0,0,35,1.0,NaN,NaN,NaN,14.0,40
4,0,0,51,2.0,NaN,NaN,NaN,3.0,57
10,0,0,98,25.0,NaN,NaN,NaN,5.0,40
11,0,0,111,1.0,NaN,NaN,NaN,1.0,57
14,0,0,153,1.0,NaN,NaN,NaN,2.0,45
18,0,0,306,1.0,NaN,NaN,NaN,1.0,45
22,0,0,482,2.0,NaN,NaN,NaN,3.0,73
23,0,0,484,4.0,NaN,NaN,NaN,4.0,73


In [42]:
training_data.to_csv('input/training_data_0.csv', index=False)

In [48]:
#drop rows with nan
training_data.dropna(inplace=True)
print(training_data.shape)

(92390, 9)


In [43]:
#add item category to data
def get_categ_id_for_row(row):
    return get_categ_id_for_item(row['item_id'])

#monthly_totals.apply(lambda row: get_categ_id_for_row(row), axis=1)
training_data['categ_id'] = training_data.apply(lambda row: get_categ_id_for_row(row), axis=1)
training_data.head()

,date_block_num,shop_id,item_id,item_cnt_day,item_cnt_tm1,item_cnt_tm2,item_cnt_tm12,item_cnt_tp1,categ_id
0,0,0,32,6.0,-1.0,-1.0,-1.0,10.0,40
1,0,0,33,3.0,-1.0,-1.0,-1.0,3.0,37
2,0,0,35,1.0,-1.0,-1.0,-1.0,14.0,40
3,0,0,43,1.0,-1.0,-1.0,-1.0,0.0,40
4,0,0,51,2.0,-1.0,-1.0,-1.0,3.0,57


In [49]:
training_data.to_csv('input/training_data_0_categ.csv', index=False)

In [45]:
#change date_block_num
def get_year_from_date_block_num(date_block_num):
    if date_block_num < 12:
        return 2013
    elif date_block_num < 24:
        return 2014
    else:
        return 2015

def get_month_from_date_block_num(date_block_num):
    return int(date_block_num % 12)

print('date block: 33 =' , get_year_from_date_block_num(33), get_month_from_date_block_num(33))


date block: 33 = 2015 9


In [43]:
def get_year_for_row(row):
    return get_year_from_date_block_num(row['date_block_num'])

monthly_totals['year'] = monthly_totals.apply(lambda row: get_year_for_row(row), axis=1)
monthly_totals.head()

,date_block_num,shop_id,item_id,item_cnt_day,categ_id,year
0,0,0,32,6.0,40,2013
1,0,0,33,3.0,37,2013
2,0,0,35,1.0,40,2013
3,0,0,43,1.0,40,2013
4,0,0,51,2.0,57,2013


In [46]:
def get_month_for_row(row):
    return get_month_from_date_block_num(row['date_block_num'])

monthly_totals['month'] = monthly_totals.apply(lambda row: get_month_for_row(row), axis=1)
monthly_totals.head()

,date_block_num,shop_id,item_id,item_cnt_day,categ_id,year,month
0,0,0,32,6.0,40,2013,0
1,0,0,33,3.0,37,2013,0
2,0,0,35,1.0,40,2013,0
3,0,0,43,1.0,40,2013,0
4,0,0,51,2.0,57,2013,0
